In [1]:
from dual import DualTensor, DualNumber, NeuralNetwork, Layer
import numpy as np
import numpy.typing as npt
from PIL import Image
from datasets import load_dataset
from albumentations import Resize


DualTensor(real=[[1, 2], [0, 0]])

DualTensor(
[[1.0 + 0.0ε 2.0 + 0.0ε]
 [0.0 + 0.0ε 0.0 + 0.0ε]]
)

In [2]:
def transform_image(image: Image) -> npt.NDArray:
    image = np.array(image) # to numpy
    image = Resize(width=12, height=12)(image=image)["image"] # resize
    image = image / 255 # normalize
    image = image.flatten() # flatten
    return image

def transform_label(label: int) -> int:
    result = np.zeros(10)
    result[label] = 1
    return result

def transform(batch):
    batch["input"] = [transform_image(image) for image in batch["input"]]
    batch["label"] = [transform_label(label) for label in batch["label"]]
    return batch


ds = load_dataset("mnist", keep_in_memory=True).with_transform(transform=transform).rename_column(original_column_name="image", new_column_name="input")
dstrain = ds["train"]

In [ ]:
def relu(x):
    return np.maximum(x, 0)

def se(x, y):
    return np.sqrt(np.sum((x - y) ** 2))

def cross_entropy(x, y):
    return -1*np.sum(y * np.log(x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

model = NeuralNetwork(input_size=12*12, loss_function=cross_entropy)
model.add_layer(n_neurons=20, activation_function=relu, initialize="He")
model.add_layer(n_neurons=10, activation_function=softmax, initialize="Xavier")

model.train(data=ds["train"],
            learning_rate=0.1,
            epochs=1,
            validation_data=ds["test"])

print(model.accuracy(ds["test"]))


EPOCH 1:  23%|██▎       | 13712/60000 [2:05:01<6:47:13,  1.89it/s, loss=0.396, validation_accuracy=0.875]  

In [6]:
print(model.accuracy(ds["test"]))

0.8917
